In [1]:
import numpy as np
import scipy.stats as sps

In [2]:
K = 80
S_0 = 76.9
sigma_1 = 0.01
sigma_2 = 0.05
alpha = 0.02
N = 100
tenor = 3

times = np.ones(N).cumsum() * tenor / N

In [3]:
def rho(i, j):
    firstTerm = sigma_1 **2 / 2 / alpha
    firstTerm *= np.exp(-alpha * np.abs(times[i] - times[j])) - np.exp(-alpha * (times[i] + times[j]))
    
    secondTerm = min(times[i], times[j])
    secondTerm += (np.exp(-alpha * np.abs(times[i] - times[j])) - np.exp(-alpha * (times[i] + times[j]))) \
        / 2 / alpha
    secondTerm -= (1 + np.exp(-alpha * np.abs(times[i] - times[j])) - np.exp(-alpha * times[i]) - np.exp(-alpha * times[j])) \
        / alpha
    secondTerm *= sigma_2**2
    return firstTerm + secondTerm
                                                  
def getRhoMatrix():
    rhoMatrix = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            rhoMatrix[i, j] = rho(i, j)
    return rhoMatrix

def asianCall(): 
    discount = np.exp(-alpha * tenor)  
    firstTerm = -K * sps.norm.cdf(y_2) + U_1 * sps.norm.cdf(y_1)

    a = (rhoMatrix.sum() / N**2)**2
    b = 0
    for i in range(N):
        b += (np.repeat([rhoMatrix[i]], N, axis=0) * rhoMatrix).sum()     
    b = 2 * b / N**3
    secondTerm = (p + dp / 2) * K * (a - b)
    
    return  (firstTerm + secondTerm) #* discount

def asianPut():
    discount = np.exp(-alpha * tenor)  
    put = asianCall() + K - S_0 / discount
    return put #* discount

In [4]:
rhoMatrix = getRhoMatrix()

U_1 = S_0
U_2 = (S_0 / N)**2 * np.exp(rhoMatrix).sum()

y = np.log(K) # Ok   
y_1 = (np.log(U_2) / 2 - y) / np.sqrt(np.log(U_2) - 2 * np.log(U_1))
y_2 = (2 * np.log(U_1) - np.log(U_2) / 2 - y) \
    / np.sqrt(np.log(U_2) - 2 * np.log(U_1))

p = np.exp(-y_2**2 / 2) / np.sqrt(2 * np.pi) / (y_1 - y_2) # Ok                                                   
dp = p * y_2 / (y_1 - y_2) # Ok   

In [5]:
asianCall()

8.977525057843475e-06

In [6]:
asianPut()

-1.6552214518131052

In [9]:
p

0.014076186191435944

In [17]:
m = 2 * np.log(U_1) - np.log(U_2) / 2
v = np.log(U_2) - 2 * np.log(U_1)
sps.norm.pdf(y, loc=m, scale=np.sqrt(v))

0.014076186191436208

Странный ответ. Должно быть примерно 0.35

In [30]:
asianCall()

5.956055442943816e-06